## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-11-14-41-41 +0000,bbc,Iran warns it will retaliate if US attacks as ...,https://www.bbc.com/news/articles/cm2jek15m8no...
1,2026-01-11-14-41-07 +0000,nyt,Israeli Police Question Top Netanyahu Aide Ove...,https://www.nytimes.com/2026/01/11/world/middl...
2,2026-01-11-14-30-46 +0000,cbc,"More than 1,000 Kyiv apartment blocks still wi...",https://www.cbc.ca/news/world/apartment-buildi...
3,2026-01-11-14-24-15 +0000,bbc,"Trump won't take Greenland by force, Mandelson...",https://www.bbc.com/news/articles/cp9j1d3zgvgo...
4,2026-01-11-14-24-00 +0000,wsj,Iran Threatens to Hit U.S. Bases if Trump Stri...,https://www.wsj.com/world/middle-east/iran-war...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,34
197,ice,20
0,iran,16
269,venezuela,15
451,minneapolis,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
144,2026-01-10-20-08-29 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,70
159,2026-01-10-17-56-09 +0000,nypost,Trump reposts Lindsey Graham’s tweet warning I...,https://nypost.com/2026/01/10/world-news/trump...,69
74,2026-01-11-03-57-21 +0000,nyt,Trump Is Briefed on Options for Striking Iran ...,https://www.nytimes.com/2026/01/10/us/politics...,67
150,2026-01-10-19-06-36 +0000,nypost,Trump admin weighing preliminary attack plans ...,https://nypost.com/2026/01/10/world-news/trump...,67
126,2026-01-10-21-44-31 +0000,nypost,"New Jersey’s Petty’s Island, now owned by Vene...",https://nypost.com/2026/01/10/us-news/new-jers...,67


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
144,70,2026-01-10-20-08-29 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...
124,66,2026-01-10-22-03-34 +0000,nypost,Renee Nicole Good seen ‘dancing’ as she blocks...,https://nypost.com/2026/01/10/us-news/renee-ni...
65,65,2026-01-11-07-27-11 +0000,nypost,Iran warns US troops and Israel will be target...,https://nypost.com/2026/01/11/world-news/iran-...
136,31,2026-01-10-20-53-00 +0000,wsj,The Defense Department has selected Owen West ...,https://www.wsj.com/politics/national-security...
152,28,2026-01-10-18-52-27 +0000,nypost,"Ilhan Omar squares off against feds, joins Min...",https://nypost.com/2026/01/10/us-news/ilhan-om...
169,28,2026-01-10-17-00-00 +0000,wsj,In an industry town that has grown increasingl...,https://www.wsj.com/politics/policy/save-calif...
123,27,2026-01-10-22-10-13 +0000,nypost,Thousands pack lefty activist group Monarca’s ...,https://nypost.com/2026/01/10/us-news/minnesot...
165,26,2026-01-10-17-23-27 +0000,nypost,US used powerful mystery weapon that brought V...,https://nypost.com/2026/01/10/world-news/us-us...
16,25,2026-01-11-13-00-00 +0000,nypost,After Marine vet Trevor Reed was freed from a ...,https://nypost.com/2026/01/11/world-news/marin...
161,23,2026-01-10-17-47-13 +0000,nypost,New NYC Council GOP leader ready to battle Mam...,https://nypost.com/2026/01/10/us-news/new-nyc-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
